In [ ]:
%pip install openai openai-agents python-dotenv pydantic GitPython PyGithub

In [ ]:
from git import Repo
from github import Github
from pydantic import BaseModel, Field
import asyncio
from dataclasses import dataclass
import os
from dotenv import load_dotenv



In [7]:
load_dotenv()
github_token = os.getenv("GITHUB_TOKEN")
openai_key = os.getenv("OPENAI_API_KEY")
project_dir = os.getcwd()

In [4]:
@dataclass
class CodeSolution(BaseModel):
    """Schema for code solution"""
    code: str = Field(..., title="Code solution")
    description: str = Field(..., title="Description of the code solution")

In [5]:
@dataclass
class CodeFeedback(BaseModel):
    feedback: str = Field(..., title="Feedback on code solution")
    refine_further: bool = Field(..., title="Flag to determine if code solution needs further refinement")

In [6]:
from agents import Agent, InputGuardrail,GuardrailFunctionOutput, Runner, ItemHelpers, TResponseInputItem, trace
from pydantic import BaseModel
import asyncio

python_coder_agent = Agent(
    model="o3-mini",
    name="Coder",
    handoff_description="Specialist agent for writing python code and test cases",
    instructions="You are a expert software engineer coding in python, who follows best practices and writes testable code. Write code based on task description by user.",
    output_type=CodeSolution,
)
reviewer_agent = Agent(
    model="o3-mini",
    name="Reviewer",
    handoff_description="Specialist agent for reviewing python code, ensures that code is well written and follows best practices",
    instructions="You are a expert code reviewer that reviews code in python. Validate that code is fully implemented, it follows user requirement, and provides feedback for correction and improvement.",
    output_type=CodeFeedback,
)

In [ ]:
def clone_repo(github_token: str, repo_url: str, repo_dir: str='') -> tuple[str, str]:
    """Clone a repo from github"""
    try:
        if not repo_dir:
            #script_dir = os.path.dirname(os.path.abspath(notebook_path))
            #repo_dir = os.path.join(script_dir, 'agent-task')
            repo_dir = os.path.join(project_dir, 'agent-task')

        if os.path.exists(repo_dir):
            print(f"A repo already exists at {repo_dir}, deleting it before cloning...")
            os.system(f"rm -rf {repo_dir}")

        print('before init github')
        g = Github(github_token)
        print('after init github')
        py_repo = g.get_repo(repo_url)
        py_branch = py_repo.get_branch('python/genai-agent-task')
        print('after get repo')
        print('Repo name : ' , py_repo.name)
        print('Repo url : ' , py_repo.url)
        print('Repo clone url : ' , py_repo.clone_url)
        local_repo = Repo.clone_from(py_repo.clone_url, repo_dir)
        #switch to the branch python/genai-agent-task
        local_repo.git.checkout(py_branch.commit.sha)
        print('after clone')
        return repo_dir, ""
    except Exception as e:
        return "", f"Repository setup failed: {str(e)}"



In [ ]:
result = clone_repo(github_token, 'prazjain/genai-scratch-repo')
print(result)

In [ ]:
def read_task_file(repo_dir: str) -> tuple[str, str]:
    """Read the task markdown file."""
    try:
        #task_path = os.path.join(repo_dir, 'tasks', 'task.md')
        task_path = os.path.join(repo_dir, 'task.md')
        with open(task_path, 'r') as f:
            return f.read(), ""
    except Exception as e:
        return "", f"Failed to read task: {str(e)}"

In [ ]:
repo_dir = result[0]
task_content, error = read_task_file(repo_dir)
print(task_content)
print(error)

In [8]:
async def generate_solution() -> CodeSolution:
    """Generate code solution based on task description."""

    max_iterations = 3

    coder_convo_history = []
    reviewer_convo_history = []

    coder_convo_history.append({"role": "user", 
                                "content": task_content })
    reviewer_convo_history.append({"role": "user",
                                   "content": f"Task description : \n{task_content}",
                             })
    print("Generating code ...")
    code_solution = await Runner.run(python_coder_agent, coder_convo_history)
    #.final_output_as(CodeSolution)
    print("Generated code.")
    for i in range(1, max_iterations+1):
        print(f"Iteration {i} of feedback..")
        reviewer_convo_history.append({"role": "user",
                                       "content": f"Please provide feedback on code solution {i} : \n{code_solution.final_output.code} ",
                                 })
        code_feedback = await Runner.run(reviewer_agent, reviewer_convo_history)
        #.final_output_as(CodeFeedback)
        print("Received feedback")
        if code_feedback.final_output.refine_further:
            print("Need further refinement.")
            coder_convo_history.append({"role": "user",
                                        "content": f"Please improve the code based on feedback : \n{code_feedback.final_output.feedback} "})
            print("Refining code ...")
            code_solution = await Runner.run(python_coder_agent, coder_convo_history)
            #.final_output_as(CodeSolution)
            print("Refined code.")
        else:
            print("Don't need further refinement.")
            break
    return code_solution

In [ ]:
sol = await generate_solution()
print("---" * 50)
print(sol)
print("---" * 50)
print(sol.final_output.code)

In [ ]:
def create_pr(solution: CodeSolution, branch, iterations) -> bool:
    """Create a pull request"""
    try:
        print('Creating PR ...')
        print('Solution : ', solution)
        sol = solution.code
        print(sol)
        repo = Repo(repo_dir)
        print('Repo : ', repo)
        print(repo.git.status())
        print(repo.git.branch())

        # Local git operations
        branch_name = f"{branch}-agent-solution-{iterations}"
        current = repo.create_head(branch_name)
        current.checkout()
        with open(os.path.join(repo_dir, 'solution.py'), 'w') as f:
            f.write(sol)
        repo.index.add(['solution.py'])
        repo.index.commit("Agent solution")
        repo.git.push('origin', branch_name)

        # remote git 
        g = Github(github_token)
        py_repo = g.get_repo('prazjain/genai-scratch-repo')

        # Github operations to create PR
        pr = py_repo.create_pull(
            title="Add Array Products Calculator",
            body=f"Implements array products calculator with the following approach:\n\n{solution.description}",
            base=branch,
            head=branch_name
        )
        print(f"Created PR: {pr.html_url}")
        return True
    except Exception as e:
        return False

In [ ]:
status = create_pr(sol.final_output, 'python/genai-agent-task', 1)
print(status)